## pysparkhw03
第3次课的作业和第2次的课一样，只是需要采用交叉验证的方法来做，并且还要计算各种分类指标，这里要分2次实验，  
* 第1次设计1个2元分类器，
注：这里的2元分类器，指的就是2值分类，2分类。   
* 第2次设计1个多元分类器。  
注：这里的多元分类器，指的就是多值分类，多分类。


采用ML Pipelines构建一个文档分类器，需要将模型进行保存，并且加载模型后对测试样本进行预测，考查点：
1）  spark读取文件  
2）  数据清洗，考查Datasets的基本操作  
3）  构建分类器的管道，考查构建各种转换操作  
4）  读取模型，读取测试数据，并且进行模型测试  
5）  重点：自己设置交叉验证的网格参数，采用交叉验证的模型来做  
6）  重点：计算分类结果的指标  
 
数据格式：
myapp_id|typenameid|typename|myapp_word|myapp_word_all
 
其中文档ID字段为：myapp_id
其中文档类别字段为：typenameid
其中文档内容为：myapp_word_all

#### 有点不明白，语料有46个分类，为什么多分类问题要选择二元分类器？

In [1]:
import pyspark
from pyspark.sql import SparkSession
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("sparkmlhw02") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()
df1 = spark.read.csv("file:///home/ian/code/data/sparkml/doc_class.dat", sep='|', header=True)
df1.show(5)

+--------+----------+--------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|
+--------+----------+--------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|
| 1376542|         2|  action|                game|game, app, enjoy,...|
| 1376603|         2|  action|run, tap, collect...|run, tap, collect...|
| 1376792|         2|  action|                 run|run, ath, game, m...|
| 1376941|         2|  action|fight, game, play...|fight, game, play...|
+--------+----------+--------+--------------------+--------------------+
only showing top 5 rows



In [2]:
df1.count()

334500

### 查看文章分类数

In [27]:
df1.select('typenameid').distinct().count()

46

In [3]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.ml.linalg import Vector
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [4]:
tokenizer = Tokenizer(inputCol='myapp_word_all', outputCol='words')
hashingTF = HashingTF(inputCol='words', outputCol='features')
df2 = df1.withColumnRenamed('typenameid','label').withColumn('label', col('label').cast(IntegerType()))
#切分训练集和测试集,会先打乱数据集
train_set, test_set = df2.randomSplit([0.9,0.1])
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [12]:
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = ParamGridBuilder().addGrid(hashingTF.numFeatures,[10, 100, 1000]).addGrid(lr.regParam, [0.1, 0.01]).build()

### 二元分类器

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

### 多元分类器

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator2 = MulticlassClassificationEvaluator(metricName='accuracy')

In [14]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=2)

In [29]:
cv2 = CrossValidator(estimator=pipeline, evaluator=evaluator1, estimatorParamMaps=paramGrid, numFolds=2)

cv3 = CrossValidator(estimator=pipeline, evaluator=evaluator2, estimatorParamMaps=paramGrid, numFolds=2)

In [33]:
%time cvModel = cv.fit(train_set)
p = cvModel.transform(test_set)
p.show()

CPU times: user 803 ms, sys: 219 ms, total: 1.02 s
Wall time: 47.9 s
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|myapp_id|label| typename|          myapp_word|      myapp_word_all|               words|            features|       rawPrediction|         probability|prediction|
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1376542|    2|   action|                game|game, app, enjoy,...|[game,, app,, enj...|(10,[0,1,2,3,4,5,...|[-7.4660730221135...|[3.76817326402624...|      31.0|
| 1376565|    4|   arcade|          game, athi|game, athi, app, ...|[game,, athi,, ap...|(10,[0,1,2,3,4,5,...|[-7.4656338457702...|[3.09631842524217...|      43.0|
| 1376595|    4|   arcade|                play|play, easi, game,...|[play,, easi,, ga...|(10,[0,1,2,3,4,5,...|[

In [30]:
%time
cvModel2 = cv2.fit(train_set)
p2 = cvModel2.transform(test_set)
p2.show()

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 14.8 µs
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|myapp_id|label| typename|          myapp_word|      myapp_word_all|               words|            features|       rawPrediction|         probability|prediction|
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1376542|    2|   action|                game|game, app, enjoy,...|[game,, app,, enj...|(1000,[10,39,40,4...|[-7.4653634298607...|[1.03979107063494...|       2.0|
| 1376565|    4|   arcade|          game, athi|game, athi, app, ...|[game,, athi,, ap...|(1000,[39,57,59,6...|[-7.4653006328221...|[3.56702219049619...|       4.0|
| 1376595|    4|   arcade|                play|play, easi, game,...|[play,, easi,, ga...|(1000,[39,57,59,6...|[-7.46

In [44]:
%time cvModel3 = cv3.fit(train_set)
p3 = cvModel3.transform(test_set)
p3.show()

CPU times: user 845 ms, sys: 251 ms, total: 1.1 s
Wall time: 54.9 s
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|myapp_id|label| typename|          myapp_word|      myapp_word_all|               words|            features|       rawPrediction|         probability|prediction|
+--------+-----+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 1376542|    2|   action|                game|game, app, enjoy,...|[game,, app,, enj...|(1000,[10,39,40,4...|[-7.4653634298607...|[1.03979107063492...|       2.0|
| 1376565|    4|   arcade|          game, athi|game, athi, app, ...|[game,, athi,, ap...|(1000,[39,57,59,6...|[-7.4653006328221...|[3.56702219049621...|       4.0|
| 1376595|    4|   arcade|                play|play, easi, game,...|[play,, easi,, ga...|(1000,[39,57,59,6...|[-

In [31]:
evaluator.evaluate(p)

1.0

In [37]:
evaluator.evaluate(p, {evaluator.metricName: "areaUnderROC"})

1.0

In [36]:
evaluator.evaluate(p, {evaluator.metricName: "areaUnderPR"})

1.0

In [38]:
evaluator1.evaluate(p2, {evaluator.metricName: "f1"})

0.9999386223068399

In [39]:
evaluator1.evaluate(p2, {evaluator.metricName: "accuracy"})

0.9999386223068399

In [40]:
evaluator1.evaluate(p2, {evaluator.metricName: "weightedPrecision"})

0.9999386223068399

In [41]:
evaluator1.evaluate(p2, {evaluator.metricName: "weightedRecall"})

0.9999386223068399

In [45]:
evaluator2.evaluate(p3)

0.9999398496240601

## end

In [19]:
cvModel.bestModel.stages[2].numFeatures

10

In [20]:
cvModel.bestModel.stages[2].numClasses

49

In [21]:
cvModel.bestModel.stages[2].summary

In [22]:
cvModel.bestModel.stages[2].getParam('regParam')

Param(parent='LogisticRegression_4613a0ab804d5f8035c4', name='regParam', doc='regularization parameter (>= 0)')

In [24]:
cvModel.getEvaluator().evaluate(cvModel.transform(test_set))

1.0

In [8]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),[(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark.createDataFrame(scoreAndLabels, ["raw", "label"])

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="raw")
evaluator.evaluate(dataset)

0.7083333333333334

In [11]:
dataset.show()

+--------------------+-----+
|                 raw|label|
+--------------------+-----+
|           [0.9,0.1]|  0.0|
|           [0.9,0.1]|  1.0|
|           [0.6,0.4]|  0.0|
|           [0.4,0.6]|  0.0|
|           [0.4,0.6]|  1.0|
|           [0.4,0.6]|  1.0|
|[0.19999999999999...|  1.0|
+--------------------+-----+



In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder
lr = LogisticRegression()
output = ParamGridBuilder() \
        .baseOn({lr.labelCol: 'l'}) \
        .baseOn([lr.predictionCol, 'p']) \
        .addGrid(lr.regParam, [1.0, 2.0]) \
        .addGrid(lr.maxIter, [1, 5]) \
        .build()
expected = [
        {lr.regParam: 1.0, lr.maxIter: 1, lr.labelCol: 'l', lr.predictionCol: 'p'},
        {lr.regParam: 2.0, lr.maxIter: 1, lr.labelCol: 'l', lr.predictionCol: 'p'},
        {lr.regParam: 1.0, lr.maxIter: 5, lr.labelCol: 'l', lr.predictionCol: 'p'},
        {lr.regParam: 2.0, lr.maxIter: 5, lr.labelCol: 'l', lr.predictionCol: 'p'}]
len(output) == len(expected)

True

In [13]:
output

[{Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol', doc='label column name.'): 'l',
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='predictionCol', doc='prediction column name.'): 'p',
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='regParam', doc='regularization parameter (>= 0).'): 1.0,
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='maxIter', doc='max number of iterations (>= 0).'): 1},
 {Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol', doc='label column name.'): 'l',
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='predictionCol', doc='prediction column name.'): 'p',
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='regParam', doc='regularization parameter (>= 0).'): 1.0,
  Param(parent='LogisticRegression_446d98d24feabc468dc1', name='maxIter', doc='max number of iterations (>= 0).'): 5},
 {Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol

In [14]:
all([m in expected for m in output])

True

In [15]:
for m in output:
    print(m)

{Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol', doc='label column name.'): 'l', Param(parent='LogisticRegression_446d98d24feabc468dc1', name='predictionCol', doc='prediction column name.'): 'p', Param(parent='LogisticRegression_446d98d24feabc468dc1', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='LogisticRegression_446d98d24feabc468dc1', name='maxIter', doc='max number of iterations (>= 0).'): 1}
{Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol', doc='label column name.'): 'l', Param(parent='LogisticRegression_446d98d24feabc468dc1', name='predictionCol', doc='prediction column name.'): 'p', Param(parent='LogisticRegression_446d98d24feabc468dc1', name='regParam', doc='regularization parameter (>= 0).'): 1.0, Param(parent='LogisticRegression_446d98d24feabc468dc1', name='maxIter', doc='max number of iterations (>= 0).'): 5}
{Param(parent='LogisticRegression_446d98d24feabc468dc1', name='labelCol', doc='label col

In [18]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
dataset = spark.createDataFrame(
        [(Vectors.dense([0.0]), 0.0),
         (Vectors.dense([0.4]), 1.0),
         (Vectors.dense([0.5]), 0.0),
         (Vectors.dense([0.6]), 1.0),
         (Vectors.dense([1.0]), 1.0)] * 10,
        ["features", "label"])
lr = LogisticRegression()
grid = ParamGridBuilder().addGrid(lr.maxIter, [0, 1]).build()
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator,
        parallelism=2)
cvModel = cv.fit(dataset)
cvModel.avgMetrics[0]


0.5

In [19]:
evaluator.evaluate(cvModel.transform(dataset))

0.8333333333333333